extract_t2m_extr.ipynb: extract absolute maximum and minimum temperature at 2m elevation for each month for every data point in contiguous US that gro-intelligence provides crop yield for

author: demetray@princeton.edu

In [1]:
import xarray as xr
import os
import pandas as pd

In [2]:
# function for rounding to nearest 1/4
def x_rnd(x):
    return round(x*4)/4

In [3]:
# path to annual wheat yields data
path = '/Users/demetrayancopoulos/Desktop/SML310/final_project/gro_ylds/wheatf_mn_ia.xlsx'
y = pd.read_excel(path)

## Extract max monthly temp

In [4]:
# path to daily max t2m data
path = '/Users/demetrayancopoulos/Desktop/SML310/final_project/era5/daily_maxt2m'

# retrieve all files in dataset
files = []
for file in os.listdir(path):
    if file.endswith('nc'):
        files.append(os.path.join(path, file))

# open files as xarray
ds = xr.open_mfdataset(files)
        

Store monthly max t2m

https://examples.dask.org/array.html

In [5]:
cols = ['yr', 'lat', 'lon', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
temps = pd.DataFrame(index = range(0, len(y.index)), columns=cols)

for ix in y.index:
    
    lat = y.loc[ix].lat     # retrieve latitude at this entry
    lat = x_rnd(lat)       # round lat coord to nearest 0.25
    
    lon = y.loc[ix].lon     # retrieve longitude at this entry
    lon = x_rnd(lon)        # round lon coord to nearest 0.25
    
    yr = y.loc[ix].StartDate[0] + y.loc[ix].StartDate[1] + y.loc[ix].StartDate[2] + y.loc[ix].StartDate[3]
    
    # t2m at this lat/lon
    t2m = ds.mx2t.sel(latitude=lat, longitude=lon)
    
    # store monthly max t2m for this lat/lon/yr
    for i in range(1, 13):
        t = yr + '-' + str(i)
        temps.iat[ix, i+2] = t2m.sel(time=t).max().compute().values+0
    
    # store yr for this point
    col = 0
    temps.iat[ix, col] = yr
    
    # store latitude for this point
    col = 1
    temps.iat[ix, col] = lat
    
    # store longitude for this point
    col = 2
    temps.iat[ix, col] = lon

spath = '/Users/demetrayancopoulos/Desktop/SML310/final_project/monthly_max_t2m.xlsx'
temps.to_excel(spath)

temps_max = temps

## Extract min monthly temp

In [7]:
# path to daily max t2m data
path = '/Users/demetrayancopoulos/Desktop/SML310/final_project/era5/daily_mint2m'

# retrieve all files in dataset
files = []
for file in os.listdir(path):
    if file.endswith('nc'):
        files.append(os.path.join(path, file))

# open files as xarray
ds = xr.open_mfdataset(files)

In [8]:
temps = pd.DataFrame(index = range(0, len(y.index)), columns=cols)

for ix in y.index:
    
    lat = y.loc[ix].lat     # retrieve latitude at this entry
    lat = x_rnd(lat)       # round lat coord to nearest 0.25
    
    lon = y.loc[ix].lon     # retrieve longitude at this entry
    lon = x_rnd(lon)        # round lon coord to nearest 0.25
    
    yr = y.loc[ix].StartDate[0] + y.loc[ix].StartDate[1] + y.loc[ix].StartDate[2] + y.loc[ix].StartDate[3]
    
    # t2m at this lat/lon
    t2m = ds.mn2t.sel(latitude=lat, longitude=lon)
    
    # store monthly min t2m for this lat/lon/yr
    for i in range(1, 13):
        t = yr + '-' + str(i)
        temps.iat[ix, i+2] = t2m.sel(time=t).min().compute().values+0
    
    # store yr for this point
    col = 0
    temps.iat[ix, col] = yr
    
    # store latitude for this point
    col = 1
    temps.iat[ix, col] = lat
    
    # store longitude for this point
    col = 2
    temps.iat[ix, col] = lon

spath = '/Users/demetrayancopoulos/Desktop/SML310/final_project/monthly_min_t2m.xlsx'
temps.to_excel(spath)

temps_min = temps

In [9]:
temps

,yr,lat,lon,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,1979,46.5,266.25,241.072937,244.214783,252.750122,256.735016,272.420532,281.721558,284.611816,282.889252,281.723358,272.594604,263.539673,247.468185
1,1980,46.5,266.25,241.840149,247.219894,247.205429,266.477295,275.718842,281.207001,287.884064,286.825623,279.173767,270.991119,265.53775,246.674286
2,1981,46.5,266.25,243.872009,241.589828,257.856354,270.060913,274.421783,283.693939,287.346283,287.286377,278.703613,269.68338,266.987518,247.948227
3,1982,46.5,266.25,238.171051,242.468475,250.42395,255.781021,277.326416,279.974792,286.586975,281.667236,280.797455,274.074707,258.078949,252.559296
4,1983,46.5,266.25,247.291702,249.726685,261.342499,265.010742,275.256653,282.424683,285.955841,288.954498,277.296356,275.739868,262.26236,238.409027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3034,1993,42.25,264.0,251.124481,248.371063,255.849915,266.515289,276.705902,280.588745,285.472137,283.027863,275.386139,264.899689,251.173248,251.328293
3035,1994,42.25,264.0,245.112854,243.947113,262.662933,267.221985,271.851715,283.317993,285.180634,283.570496,278.059265,271.482605,262.463562,247.983566
3036,1995,42.25,264.0,248.655914,253.652802,249.555252,267.16684,274.385254,280.937439,282.529327,286.759155,273.173492,273.692169,262.932251,255.307571
3037,1996,42.25,264.0,241.173172,238.208832,252.653427,267.69751,276.397552,279.185028,283.590118,286.452423,278.475098,271.220123,257.00943,248.182266
